In [1]:
import glob
import numpy as np
import argparse

In [2]:
prediction_folder_path = "predicted_files/"
output_folder_path = "output_files/"

predictionPaths = glob.glob(prediction_folder_path+'*.txt')

In [3]:
class_names = ["specularity", "saturation", "artifact", "blur", "contrast", 
               "bubbles", "instrument", "blood"]

IoUs_path = "files/class_iou_2_0.txt"

with open(IoUs_path) as f:
    contents = f.readlines()

# class_thresholds[target_list[-1][0]][contents[counter][0]]
hard_threshold = 0.4

class_thresholds  = {}
temp_dict = {}

for i in range(len(contents)):
        
    main_class = i // 8
    sub_class = i-8*main_class
        
    content = float(contents[i].split(" ")[1][0:7])
    temp_dict[class_names[sub_class]]=content
    
    if (i%8 == 7):
        class_thresholds[class_names[main_class]]= temp_dict
        temp_dict = {}

In [4]:
def IoU(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # return the intersection over union value
    return iou

In [5]:
for predictionPath in predictionPaths:
    txt_file_name = predictionPath.split("/")[-1]
    target_list = []

    contents = np.loadtxt(predictionPath, dtype=str)    
    contents = contents[np.argsort(contents[:, 1])][::-1]
    # debug
    print(contents)
    print("---------------------------------------------------------------")
    
    while (len(contents) > 0):
        target_list.append(contents[0])
        
        # debug
        print("add: "+str(target_list[-1][0:2]))
        
        contents = np.delete(contents, 0, 0)
        
        contents_size = len(contents)
        counter = 0
        while (counter < contents_size):                        
            if (IoU(target_list[-1][2:].astype(float), contents[counter][2:].astype(float)) > class_thresholds[target_list[-1][0]][contents[counter][0]]):
                # debug
                print("IoU: "+str(IoU(target_list[-1][2:].astype(float), contents[counter][2:].astype(float))))
                print("delete: "+str(contents[counter][0:2]))
                    
                contents = np.delete(contents, counter, 0)
                counter = counter-1
                contents_size = contents_size-1
            counter = counter+1
    
    temp_detection_list = []
    for i in range(len(target_list)):
            temp_detection = target_list[i][0]+" "+ target_list[i][1]+" "+target_list[i][2]+" "+ target_list[i][3] +" "+target_list[i][4]+" "+ target_list[i][5]
            temp_detection_list.append(temp_detection)   
            
    with open(output_folder_path+txt_file_name, 'w') as f:
        for item in temp_detection_list:
            f.write("%s\n" % item)

[['blur' '0.96' '2.41470336914' '5.749557495117' '512.0' '512.0']
 ['contrast' '0.96' '227.593383789' '139.3919525146' '381.8864135742'
  '320.8430786132']
 ['blur' '0.33' '131.58454895' '86.1164093017' '424.6286621'
  '373.772338867']
 ['contrast' '0.26' '140.1119537353' '117.5495758056' '401.585'
  '341.2612304']
 ['artifact' '0.15' '492.860870361' '486.83618164' '509.87625122'
  '506.155883789']]
---------------------------------------------------------------
add: ['blur' '0.96']
add: ['contrast' '0.96']
IoU: 0.4803836328390893
delete: ['contrast' '0.26']
add: ['blur' '0.33']
add: ['artifact' '0.15']


In [16]:
boxA = np.array([353.075408935, 301.3045043945, 398.42288208, 387.420898])
boxB = np.array([358.504882, 321.795928955, 391.3056030273, 379.953216552])

print(IoU(boxA.astype(float), boxB.astype(float)))

0.4952315078467867
